# Density-based spatial clustering of applications with noise (DBSCAN) / Основанная на плотности пространственная кластеризация для приложений с шумами

---

**Источники:**


[Density-based spatial clustering of applications with noise (DBSCAN)](https://en.wikipedia.org/wiki/DBSCAN)

[Основанная на плотности пространственная кластеризация для приложений с шумами](https://ru.wikipedia.org/wiki/DBSCAN)

[DBSCAN](https://scikit-learn.org/stable/modules/clustering.html#dbscan)

[Интересные алгоритмы кластеризации, часть вторая: DBSCAN](https://habr.com/ru/post/322034/)

[]()

[]()

[]()

[]()

[]()

---

## Подготовка окружения

In [1]:
# ВНИМАНИЕ: необходимо удостовериться, что виртуальная среда выбрана правильно!

# Для MacOS/Ubuntu
# !which pip

# Для Windows
# !where pip

In [2]:
# !conda install matplotlib numpy scikit-learn seaborn -y

In [3]:
# !conda install basemap matplotlib -y

In [4]:
# !conda install -c conda-forge umap-learn -y

In [5]:
import numpy as np

np.__version__

'1.20.2'

In [6]:
import pandas as pd

pd.__version__

'1.2.4'

In [7]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

matplotlib.__version__

'3.3.4'

In [8]:
import seaborn as sns

sns.__version__

'0.11.1'

In [ ]:
import umap
from umap import UMAP

umap.__version__

In [ ]:
import sklearn

from sklearn.decomposition import PCA

from sklearn.preprocessing import QuantileTransformer

sklearn.__version__

## Описание

DBSCAN — это алгоритм кластеризации данных, который предложили Маритин Эстер, Ганс-Петер Кригель, Ёрг Сандер и Сяовэй Су в 1996.

**Подходит для данных, содержащих кластеры одинаковой плотности.**

Это алгоритм кластеризации, **основанной на плотности** — если дан набор точек в некотором пространстве, алгоритм **группирует точки, которые тесно расположены** (точки со многими близкими соседями), **помечая как выбросы** точки, которые находятся одиноко в областях с малой плотностью (ближайшие соседи которых лежат далеко).

**DBSCAN является одним из наиболее часто используемых алгоритмов кластеризации, и наиболее часто упоминается в научной литературе.**

В 2014 алгоритм получил премию "проверено временем (test of time)" (премия даётся алгоритмам, которые получили существенное внимание в теории и практике) на ведущей конференции по интеллектуальному анализу данных.


Самые важные гиперпараметры [sklearn.cluster.DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html):

- **`eps (default=0.5)`** - максимальное расстояние между двумя примерами (samples), чтобы один считался соседним с другим. Это не максимальная граница расстояний до точек в кластере. Это наиболее важный параметр DBSCAN, который нужно выбрать в соответствии с набором данных и функцией расстояния (`metric`).

- **`min_samples (default=5)`** - количество примеров (samples) или общий вес в окрестности точки, которая будет считаться базовой точкой. Сюда входит и сама точка.

Любая задача интеллектуальной обработки данных имеет проблему параметров. 

Любой параметр специфично влияет на алгоритм. Для алгоритма DBSCAN нужны параметры `eps`  и `min_samples`.

**В идеале, значение `eps`  определяется решаемой задачей (например, физические расстояния), а `min_samples` определяет тогда минимальный желаемый размер кластера.**

**OPTICS** можно рассматривать как **обобщение DBSCAN**, в котором параметр **`eps`  заменяется максимальным значением**, наиболее воздействущим на эффективность. **`min_samples` тогда становится минимальным размером кластера**. 

Хотя алгоритм **OPTICS** существенно проще в области выбора параметров, чем DBSCAN, его результаты труднее использовать, так как он обычно даёт иерархическую кластеризацию вместо простого разделения, которое даёт DBSCAN.

### Рекомендации по выбору гиперпараметров `eps`  и `min_samples`

- **`min_samples`**

    - Минимальное значение `min_samples` может быть получено из размерности `D` (например, 2D, 3D, 8D = количество признаков) набора данных как $min\_samples \geqslant D +1$. 
    
    - Низкое значение $min\_samples=1$ не имеет смысла, так как тогда любая точка будет кластером.
    
    - Для $min\_samples \leqslant 2$ результат будет тем же самым, что и иерархическая кластеризация с метрикой единичного соединения с отсечением дендрограммы на высоте `eps`.
    
    - **`min_samples` должен быть равным как минимум 3**. 
    
    - Для наборов данных **с шумами бо́льшие значения `min_samples` обычно лучше**, и дают более существенные кластеры. 
    
    - Эмпирика показывает, что может быть использовано значение $min\_samples = 2 * D$, но может оказаться необходимым **выбор большего значения для больших наборов данных**, для данных с шумом или для данных, содержащих много дубликатов.
 

- **`eps`**

    - Значение `eps` может быть выбрано с помощью графа k-расстояний, вычерчивая расстояние `k` ($k = min\_samples - 1$) ближайшему соседу в порядке от большего к меньшему.
    
    - **Хорошие значения `eps` те, где график имеет "изгиб"**.
    
    - **Если `eps` выбрана слишком малыми, большая часть данных не будет кластеризована, а для слишком больших значений `eps`  кластеры будут сливаться и большинство объектов окажутся в одном кластере**.
    
    - Обычно **малые значения `eps`  предпочтительнее** и опыт показывает, что только небольшая доля точек должна быть с этим расстоянием друг от друга.
    
    - Альтернативно, может быть использован график OPTICS для выбора `eps`, но тогда и сам алгоритм OPTICS может быть использован для кластеризации.
    
- **`metric`**: 

    - Выбор функции расстояния сильно связан с выбором `eps` и имеет большое влияние на результаты.
    
    - Обычно сначала необходимо определить обоснованные меры похожести набора данных, прежде чем выбирать параметр `eps`.
    
    - Нет оценок для этого параметра, но **функции расстояния следует выбирать согласно набору данных**.
    
    - Например, для географических данных, расстояние по дуге большого круга часто будет хорошим выбором.

### Преимущества

- DBSCAN **не требует указать число кластеров** в данных априори в отличие от метода k-средних.

- DBSCAN может найти **кластеры произвольной формы**. 
    - DBSCAN может найти даже кластеры полностью окружённые (но не связанные с) другими кластерами.
    - Благодаря параметру `min_samples` уменьшается так называемый эффект одной связи (связь различных кластеров тонкой линией точек).


- DBSCAN имеет понятие шума и **устойчив к выбросам**.

- DBSCAN требует лишь двух параметров и большей частью **нечувствителен к порядку точек** в наборе данных.
    - Однако, точки, находящиеся на границе двух различных кластеров могут оказаться в другом кластере, если изменить порядок точек, а назначение кластеров единственно с точностью до [изоморфизма](https://ru.wikipedia.org/wiki/%D0%98%D0%B7%D0%BE%D0%BC%D0%BE%D1%80%D1%84%D0%B8%D0%B7%D0%BC).


- DBSCAN разработан для применения с базами данных, которые позволяют ускорить запросы в диапазоне значений, например, с помощью [R*-дерева](https://ru.wikipedia.org/wiki/R*-%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).

- Параметры `min_samples` и `eps`  могут быть установлены экспертами в рассматриваемой области, если данные хорошо понимаются.

### Недостатки

- DBSCAN **не полностью однозначен — краевые точки, которые могут быть достигнуты из более чем одного кластера, могут принадлежать любому из этих кластеров, что зависит от порядка просмотра точек**.
    - Для большинства наборов данных эти ситуации возникают редко и имеют малое влияние на результат кластеризации — основные точки и шум DBSCAN обрабатывает однозначно. 
    - Существует версия DBSCAN, которая трактует краевые точки как шум и тем самым достигается полностью однозначный результат, а также более согласованная статистическая интерпретация связных по плотности компонент.

- Качество DBSCAN **зависит от измерения расстояния (`metric`)**.
    - Наиболее часто используемой метрикой расстояний является евклидова метрика. 
    - Особенно для кластеризации **данных высокой размерности евклидова метрика может оказаться почти бесполезной** ввиду так называемого "[проклятия размерности](https://ru.wikipedia.org/wiki/%D0%9F%D1%80%D0%BE%D0%BA%D0%BB%D1%8F%D1%82%D0%B8%D0%B5_%D1%80%D0%B0%D0%B7%D0%BC%D0%B5%D1%80%D0%BD%D0%BE%D1%81%D1%82%D0%B8)", что делает трудным делом нахождение подходящего значения `eps`. 
    - Этот эффект, однако, присутствует в любом другом алгоритме, основанном на евклидовом расстоянии.

- DBSCAN не может хорошо кластеризовать наборы данных **с большой разницей в плотности**, поскольку не удается выбрать приемлемую для всех кластеров комбинацию `min_samples` и `eps`.

- Если данные и масштаб не вполне хорошо поняты, **выбор осмысленного порога расстояния `eps`  может оказаться трудным**.

## Загрузка данных

[Источник (custDatasets)](https://www.kaggle.com/gangliu/custdatasets).

In [ ]:
df = pd.read_csv('./../../data/Cust_Segmentation.csv', index_col=0)
df

## Анализ данных

См. лекцию [02_pca](../../08_modeling_ml_demensionality_reduction/lectures/02_pca.ipynb)

## Подготовка данных

In [ ]:
df['Defaulted'].fillna(0, inplace=True)
df['Defaulted'] = df['Defaulted'].astype(int)
df['Defaulted'].isna().sum()

In [ ]:
num_cols = df.select_dtypes(include=np.number).columns.tolist()
norm_trans = QuantileTransformer(output_distribution='normal', n_quantiles=100)
df_norm = pd.DataFrame(norm_trans.fit_transform(df[num_cols]), columns=num_cols)
df_norm.hist(bins=30, figsize=(15, 5))
plt.tight_layout()

## 3-D PCA

In [ ]:
pca_3 = PCA(n_components=3)
df_pca_3 = pd.DataFrame(pca_3.fit_transform(df_norm))

In [ ]:
%matplotlib widget
from mpl_toolkits.mplot3d.axes3d import Axes3D
fig = plt.figure(figsize=(5, 5))
ax = Axes3D(fig, azim=-135, elev=35)

ax.scatter(df_pca_3[0], df_pca_3[1], df_pca_3[2],
           alpha=0.3)

In [ ]:
%matplotlib inline

## 3-D UMAP

In [ ]:
umap_3 = UMAP(n_components=3)
df_umap_3 = pd.DataFrame(umap_3.fit_transform(df_norm))

In [ ]:
%matplotlib widget

from mpl_toolkits.mplot3d.axes3d import Axes3D
fig = plt.figure(figsize=(5, 5))
ax = Axes3D(fig, azim=25, elev=35)

ax.scatter(df_umap_3[0], df_umap_3[1], df_umap_3[2], alpha=0.3)

In [ ]:
%matplotlib inline

## Выбор `min_samples`

In [ ]:
df_norm

In [ ]:
min_samples = len(df_norm.columns) + 1
min_samples

## Выбор `eps`

In [ ]:
from sklearn.neighbors import NearestNeighbors

# рассчитать среднее расстояние между каждой точкой 
# в наборе данных и ее min_samples ближайшими соседями 
neighbors = NearestNeighbors(n_neighbors=min_samples-1)
neighbors_fit = neighbors.fit(df_norm)
distances, indices = neighbors_fit.kneighbors(df_norm)

In [ ]:
# сортировка значений расстояний по возрастанию и построение графика
distances = np.sort(distances, axis=0)

In [ ]:
distances = distances[:, 1]

In [ ]:
# оптимальное значение для epsilon будет найдено 
# в точке максимальной кривизны
plt.plot(distances)
plt.xlabel('distances')
plt.ylabel('epsilon')
plt.grid(True)

In [ ]:
epsilon=1.7

## Построение модели

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
db = DBSCAN(eps=epsilon, min_samples=min_samples).fit(df_norm)
labels = db.labels_
labels

In [ ]:
sns.countplot(x=labels)

In [ ]:
clusters = np.unique(labels)
clusters

## Анализ результатов

### 3-D PCA

In [ ]:
%matplotlib widget
from mpl_toolkits.mplot3d.axes3d import Axes3D
fig = plt.figure(figsize=(5, 5))
ax = Axes3D(fig, azim=-125, elev=35)

ax.scatter(df_pca_3[0], df_pca_3[1], df_pca_3[2], 
           c=labels, 
           alpha=0.3, 
           cmap='hsv', 
           s=60)

In [ ]:
%matplotlib inline

### 3-D UMAP

In [ ]:
%matplotlib widget

from mpl_toolkits.mplot3d.axes3d import Axes3D    
fig = plt.figure(figsize=(5, 5))
ax = Axes3D(fig, azim=142, elev=35)

ax.scatter(df_umap_3[0], df_umap_3[1], df_umap_3[2], 
           c=labels, 
           alpha=0.3, 
           cmap='hsv', 
           s=60)

In [ ]:
%matplotlib inline

### Анализ "представителей" кластеров

In [ ]:
df["Cluster"] = labels
df

In [ ]:
df.groupby('Cluster').mean().round(2)

In [ ]:
df_clusters = {}
cluster_examples = pd.DataFrame()

for c in clusters:
    print(f'Cluster = {c}')
    df_clusters[c] = df[df.Cluster == c]
    cluster_examples = cluster_examples.append(df[df.Cluster == c].head(1))
    display(cluster_examples)
    display(df_clusters[c])
    display(df_clusters[c].describe())
    print('\n', '=' * 100, '\n')

#### 2-D

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3))

sns.scatterplot(x=df.Edu, 
                y=df.Defaulted, 
                size=df.Income, 
                sizes=(10, 450), 
                hue=df.Cluster, 
                palette='hsv', 
                alpha=0.3,
                ax=ax)

ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 10))

sns.scatterplot(x=df.Edu, 
                y=df['Years Employed'], #df.Defaulted, 
                size=df.Defaulted, 
                sizes=(250, 50), 
                hue=df.Cluster, 
                palette='hsv', 
                alpha=0.3,
                ax=ax)

ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

#### 3-D

In [ ]:
%matplotlib widget
from mpl_toolkits.mplot3d.axes3d import Axes3D
fig = plt.figure(figsize=(5, 5))
ax = Axes3D(fig, azim=-45, elev=25)


ax.scatter(df.Edu, df.Defaulted, df['Years Employed'], #df.Income, 
           c=df.Cluster,
           alpha=0.3, 
           s=60, 
           cmap='hsv')

ax.set_xlabel('Education')
ax.set_ylabel('Defaulted')
ax.set_zlabel('Years Employed')
# ax.set_zlabel('Income')


ax.scatter(cluster_examples.Edu, cluster_examples.Defaulted, cluster_examples['Years Employed'], marker='*',
           c="white", alpha=1, s=500, edgecolor='k')

for row, c in enumerate(cluster_examples.Cluster):
    ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                s=400, edgecolor='k')

In [ ]:
%matplotlib inline